# NEOSSAT Tutorial - 01 - Extracting the data and visualization
***

**Tutorial :** This tutorial provides a step to step guide to extract data from the the Canadian astronomy data centre (CADC) and the open data portal of the Canadian Space Agency. </br>
**Mission and Instrument :** NEOSSAT  
**Astronomical Target :** Detecting and tracking near earth objects      
**System Requirements :** Python 3.9 or later  
**Tutorial Level :** Basic  

For more information on on the NEOSSAT space telescope and the FITS files please consult NEOSSat FITS Image User’s Guide via the following link: https://donnees-data.asc-csa.gc.ca/users/OpenData_DonneesOuvertes/pub/NEOSSAT/Supporting%20Documents/CSA-NEOSSAT-MAN-0002_FITS_IMAGE_UGUIDE-v4-00.pdf

**Extracting data from the open data portal of the Canadian Space Agency (CSA):**: The Canadian Space Agency has a dedicated page to NEOSSAT datasets on its open data and information webapge which can be found via the following link: https://donnees-data.asc-csa.gc.ca/en/dataset/9ae3e718-8b6d-40b7-8aa4-858f00e84b30

**Extracting data from the Canadian Astronomy Data Centre (CADC)**: CADC recoemmnds the installation of CADCdata package for usage in python. Documentation on how to access the library is available with the pydoc cadcdata. For more information visit: https://www.cadc-ccda.hia-iha.nrc-cnrc.gc.ca/en/doc/data/#library



## Option 1 - Extracting NEOSSAT data from the Canadian Space Agency (CSA)
Open the following FTP link on file explorer and click on the 'Astro' folder to get access to the .fits files of NEOSSAT from 2017 to 2023: ftp://ftp.asc-csa.gc.ca/users/OpenData_DonneesOuvertes/pub/NEOSSAT/

With the Canadian Space Agency dataset you can filter the NEOSSAT data based on the year and day of year. You can filter the data based on the sky coordinates (i.e. CMD value on the header data unit of the .fits files) yet the process might take longer than usual to get completed. 

### Importing required libraries

In [ ]:
from astropy.visualization import astropy_mpl_style
from astropy.io import fits
import matplotlib.pyplot as plt
from ftplib import FTP
import os
import re
from astropy.coordinates import SkyCoord
import astropy.units as u

In [ ]:
#Define a local directory path to download the .fits files. 
local_directory = 'C:/Users/rsheikholmolouki/Desktop/NEOSSAT_CSA/Part 1/'

In [ ]:
#initialize the server
ftp_host = 'ftp.asc-csa.gc.ca'
ftp = FTP (ftp_host)
ftp.login ()

In [ ]:
#Switch to the NEOSSAT directory path
ftp_directory = '/users/OpenData_DonneesOuvertes/pub/NEOSSAT/ASTRO/'
ftp.cwd(ftp_directory)

In [ ]:
#Print out the NEOSSAT collection folders
content = ftp.nlst()
years = [item for item in content if "." not in item]
print ("Neossat_collection:")
for year in years:
    print (year)

In [ ]:
#Filtering the .fits files in mind based on year 
selected_year = '2021' #change the year to the ones in the collection above.
CMD = 'RA=3.802 DEC=-0.750 ROLL=1.745' # For filtering based on the sky coordinates

In [ ]:
#Code iterating over each subdirectory to check the header for the CMD criteria
def filter_fits_match (ftp, directory):
    matching_fits_files = []
    #files_in_directory = [item for item in ftp.nlst(directory) if '.' not in item]
    subfolders_in_directory = [item for item in ftp.nlst(directory) if '.' not in item]
    files_in_directory = []
    for subfolder in subfolders_in_directory:
        #print (subfolder)
        tmpRawFiles = [item for item in ftp.nlst(subfolder)]
        for tmp_file in tmpRawFiles:
            tmp_file = subfolder + '/' + tmp_file
        files_in_directory = files_in_directory + tmpRawFiles
    #print (len(files_in_directory))
    for filename in files_in_directory:
        print ('File to parse: ' + filename)
        if filename.lower().endswith('.fits'):
            try:
                print("ftp://{ftp_host}/{ftp_directory}/{filename}")
                #remote_filepath = f"ftp://{ftp_host}/{ftp_directory}/{directory}/{filename}"
                remote_filepath = f"ftp://{ftp_host}/{ftp_directory}/{filename}"
                hdul = fits.open(remote_filepath, cache =True)
                #header = fits.getheader(remote_filepath, cache=True)
                #cmd_value = hdul[0].header.get('CMD','').strip()
                print(f' File: {filename}, CMD value: {cmd_value}')
                #cmd_value = header.get (remote_filepath, 'CMD').strip()
                hdul.close()
            
                if cmd_value == CMD:##CMD Header data unit (HDU) commanded spacecraft pointing, and celestial coordinates, in Science CCD frame. 
                    matching_fits_file.append((directory, filename))
            except Exception as ex:
                print (f" Error processing file {filename}: {str(ex)}")
    subdirectories = [item for item in ftp.nlst(directory) if '.' not in item]
    for subdir in subdirectories:
        matching_fits_files.extend(filter_fits_match(ftp, f'{directory}/{subdir}'))
    return matching_fits_files

In [ ]:
matching_files = filter_fits_match(ftp, selected_year)
print ("Matching .fits files:")
for directory, filename in matching_files:
    print (f' {directory}/{filename}')

In [ ]:
for directory, filename in matching_files:
    remote_filepath = f'{directory}/{filename}'
    local_filepath = os.path.join(local_directory, filename)
    with open (local_filepath, 'wb') as local_file:
        ftp.retrbinary(f'RETR {remote_filepath}', local_file.write)

In [ ]:
ftp.quit()

### You can also download only one file from the FTP file and then visualize it

In [ ]:
#Download the .fits file of choice in a folder. 
#In this section the following fits file was downloaded: 2019 folder >> 292 >> NEOS_SCI_2019292034200.fits
source = 'C:/Users/rsheikholmolouki/Desktop/NEOSSAT_CSA/' #folder
filename = source + 'NEOS_SCI_2019292034200.fits' #name of the specific .fits file

In [ ]:
img_header = fits.getheader(filename)
print(img_header)

### Visualizing the FITS image data

In [ ]:
#Viusalize the fits file
image_data = fits.getdata(filename, ext =0)
plt.style.use(astropy_mpl_style)
plt.figure()
plt.axis ('off')
plt.imshow(image_data, cmap='gray')
plt.show()

## Option 2 - Extracting NEOSSAT data from the Canadian Astronomy Data Centre (CADC)

With CADC datasets, in addition to filtering the data based on time, you can also filter through NEOSSAT data based on the sky coordinates, time exposure and more! 

### Importing required libraries

In [ ]:
from astroquery.cadc import Cadc
from astropy import units as u
from astropy.coordinates import SkyCoord
from astropy.visualization import astropy_mpl_style
from astropy.io import fits
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
#Initialize the CADC client 
cadc = Cadc()

#Print information about availbale collection 
for collection, details in sorted(cadc.get_collections().items()):
   print(f'{collection} : {details}')

In [ ]:
#Define the target coordinates and search radius based on your preference
coords = SkyCoord(240, -30, unit = 'deg') #RA, DEC, Unit
radius = 2* u.deg

#Query CADC for data within the specificied region and collection 
results = cadc.query_region(coords,radius, collection = 'NEOSSAT')
print (results)

In [ ]:
#Filter the query results to select data with a specifc 'time_exposure'. 
# In this example 'time exposure' greater than 50 is selected

filtered_results = results[results['time_exposure'] > 50.0]

#Access data points from the filtered results 
print(filtered_results['time_exposure'][100])
print(filtered_results['position_dimension_naxis2'][100])
print(filtered_results['position_dimension_naxis1'][100])
print(filtered_results['instrument_keywords'][100])
print(filtered_results['metaRelease'][100])

In [ ]:
#Get a list of image URLs based on the filtered results
image_list = cadc.get_image_list(filtered_results, coords, radius)

#Print the number of images int he image_list 
print(len(image_list))

#print the last URL in the image_list 
print(image_list[-1])

#Get the filename of the 100th image from the image_list 
filename = image_list[100]
print(filename)

### Visualizing the FITS image data

In [ ]:
#Read the FITS image data from the file
image_data = fits.getdata(filename, ext =0)

In [ ]:
#Show the image file
plt.style.use(astropy_mpl_style)
plt.figure()
plt.axis ('off')
plt.imshow(image_data, cmap='gray')
plt.show()

 ==> Coming soon: Creating a sky map with NEOSSAT data and processing its images.